In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

/bin/bash: nvcc: command not found
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.6.0 True
0.22.1


In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.rc('font', size=12) 
plt.rc('figure', figsize = (12, 5))

# Settings for the visualizations
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2,'font.family': [u'times']})

import pandas as pd
pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 50)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [4]:
from torchvision import datasets, models, transforms
import torch.optim as optim

if torch.cuda.is_available():
    torch.cuda.init()
    print(torch.cuda.current_device())
    print(torch.cuda.device_count())
    print(torch.cuda.memory_allocated())
    print(torch.cuda.memory_cached())
    cuda = torch.device('cuda')
    cuda0 = torch.device('cuda:0')
    print(cuda0)

0
1
0
0
cuda:0


/home/user/anaconda3/envs/python37/lib/python3.7/site-packages/torch/cuda/memory.py:346: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


In [5]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [6]:
torch.cuda.empty_cache()

In [7]:
from mmseg.apis import init_segmentor, inference_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

In [8]:
import os.path as osp
import numpy as np
from PIL import Image
import skimage
from skimage import io
print(skimage.__version__)

0.18.3


In [9]:
import mmcv

In [10]:
classes = ['background', 'human']
palette = [[0,0,0], [255,0,0]]
class_weight = [1.0,20.0] # 0.05 human pixels

In [11]:
import os.path as osp
import mmcv
import numpy as np
from PIL import Image

from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset


@DATASETS.register_module()
class ADE20KDataset01(CustomDataset):
    """ADE20K dataset.
    In segmentation map annotation for ADE20K, 0 stands for background, which
    is not included in 150 categories. ``reduce_zero_label`` is fixed to True.
    The ``img_suffix`` is fixed to '.jpg' and ``seg_map_suffix`` is fixed to
    '.png'.
    """
    CLASSES = classes
    PALETTE = palette

    def __init__(self, **kwargs):
        super(ADE20KDataset01, self).__init__(
            img_suffix='.jpg',
            seg_map_suffix='.png',
            reduce_zero_label=False,
            **kwargs)
        assert osp.exists(self.img_dir)

    def results2img(self, results, imgfile_prefix, to_label_id, indices=None):
        if indices is None:
            indices = list(range(len(self)))

        mmcv.mkdir_or_exist(imgfile_prefix)
        result_files = []
        for result, idx in zip(results, indices):

            filename = self.img_infos[idx]['filename']
            basename = osp.splitext(osp.basename(filename))[0]

            png_filename = osp.join(imgfile_prefix, f'{basename}.png')

            output = Image.fromarray(result.astype(np.uint8))
            output.save(png_filename)
            result_files.append(png_filename)

        return result_files

    def format_results(self,
                       results,
                       imgfile_prefix,
                       to_label_id=True,
                       indices=None):
        
        if indices is None:
            indices = list(range(len(self)))

        assert isinstance(results, list), 'results must be a list.'
        assert isinstance(indices, list), 'indices must be a list.'

        result_files = self.results2img(results, imgfile_prefix, to_label_id,
                                        indices)
        return result_files

In [12]:
from mmcv import Config
cfg = Config.fromfile('../configs/stdc/stdc1_512x1024_80k_cityscapes.py')

In [13]:
data_root = '../data/ade/ADEChallengeData2016'
img_dir = 'images-binary'
ann_dir = 'binary-labels'

In [14]:
channels=(32, 64, 256, 512, 1024)

In [15]:
decode_head_loss_weight = 1.0
auxiliary_head_loss_weight = decode_head_loss_weight

In [16]:
norm_config_type = 'BN'

In [17]:
from mmseg.apis import set_random_seed
# Since we use ony one GPU, BN is used instead of BN
cfg.norm_cfg = dict(type=norm_config_type, requires_grad=True)  # Segmentation usually uses SyncBN
# model settings
cfg.model = dict(
    type='EncoderDecoder',
    pretrained=None,
    backbone=dict(
        type='STDCContextPathNet',
        backbone_cfg=dict(
            type='STDCNet',
            stdc_type='STDCNet2',
            in_channels=3,
            channels=channels,
            bottleneck_type='cat',
            num_convs=4,
            norm_cfg=cfg.norm_cfg,
            act_cfg=dict(type='ReLU'),
            with_final_conv=False),
        last_in_channels=(1024, 512),
        out_channels=128,
        ffm_cfg=dict(in_channels=384, out_channels=256, scale_factor=4)),
    decode_head=dict(
        type='FCNHead',
        in_channels=256,
        channels=256,
        num_convs=1,
        num_classes=len(classes),
        in_index=3,
        concat_input=False,
        dropout_ratio=0.1,
        norm_cfg=cfg.norm_cfg,
        align_corners=False,
        sampler=dict(type='OHEMPixelSampler', thresh=0.7, min_kept=10000),
        loss_decode=[
                dict(
                    type='CrossEntropyLoss',
                    loss_name='loss_ce',
                    use_sigmoid=False,
                    loss_weight=decode_head_loss_weight, 
                    #class_weight=class_weight
                ),
                dict(
                    type='DiceLoss',
                    loss_name='loss_dice',
                    loss_weight=decode_head_loss_weight, 
                    class_weight=class_weight
                )
            ]
    ),
    auxiliary_head=[
        dict(
            type='FCNHead',
            in_channels=128,
            channels=64,
            num_convs=1,
            num_classes=len(classes),
            in_index=2,
            norm_cfg=cfg.norm_cfg,
            concat_input=False,
            align_corners=False,
            sampler=dict(type='OHEMPixelSampler', thresh=0.7, min_kept=10000),
            loss_decode=[
                dict(
                    type='CrossEntropyLoss',
                    loss_name='loss_ce',
                    use_sigmoid=False,
                    loss_weight=decode_head_loss_weight, 
                    #class_weight=class_weight
                ),
                dict(
                    type='DiceLoss',
                    loss_name='loss_dice',
                    loss_weight=decode_head_loss_weight, 
                    class_weight=class_weight
                )
            ]
        ),
        dict(
            type='FCNHead',
            in_channels=128,
            channels=64,
            num_convs=1,
            num_classes=len(classes),
            in_index=1,
            norm_cfg=cfg.norm_cfg,
            concat_input=False,
            align_corners=False,
            sampler=dict(type='OHEMPixelSampler', thresh=0.7, min_kept=10000),
            loss_decode=[
                dict(
                    type='CrossEntropyLoss',
                    loss_name='loss_ce',
                    use_sigmoid=False,
                    loss_weight=decode_head_loss_weight, 
                    #class_weight=class_weight
                ),
                dict(
                    type='DiceLoss',
                    loss_name='loss_dice',
                    loss_weight=decode_head_loss_weight, 
                    class_weight=class_weight
                )
            ]
        ),
        dict(
            type='STDCHead',
            in_channels=256,
            channels=64,
            num_convs=1,
            num_classes=len(classes),
            boundary_threshold=0.1,
            in_index=0,
            norm_cfg=cfg.norm_cfg,
            concat_input=False,
            align_corners=False,
            loss_decode=[
                dict(
                    type='CrossEntropyLoss',
                    loss_name='loss_ce',
                    use_sigmoid=True,
                    loss_weight=decode_head_loss_weight, 
                    #class_weight=class_weight
                ),
                dict(
                    type='DiceLoss',
                    loss_name='loss_dice',
                    loss_weight=decode_head_loss_weight, 
                    class_weight=class_weight
                )
            ]
        ),
    ],
    # model training and testing settings
    train_cfg=dict(),
    test_cfg=dict(mode='whole'))

In [18]:
############### start dataset 
cfg.dataset_type = 'ADE20KDataset01'  # Dataset type, this will be used to define the dataset.
cfg.data_root = data_root  # Root path of data.
cfg.img_norm_cfg = dict(  # Image normalization config to normalize the input images.
    mean=[123.675, 116.28, 103.53],  # Mean values used to pre-training the pre-trained backbone models.
    std=[58.395, 57.12, 57.375],  # Standard variance used to pre-training the pre-trained backbone models.
    to_rgb=True)  # The channel orders of image used to pre-training the pre-trained backbone models.
cfg.crop_size = (512, 512)  # The crop size during training.

cfg.train_pipeline = [  # Training pipeline.
    dict(type='LoadImageFromFile'),  # First pipeline to load images from file path.
    dict(type='LoadAnnotations', reduce_zero_label=False),  # Second pipeline to load annotations for current image.
    dict(type='Resize',  # Augmentation pipeline that resize the images and their annotations.
        img_scale=(2048, 512),  # The largest scale of image.
        ratio_range=(0.5, 2.0)), # The augmented scale range as ratio.
    dict(type='RandomCrop',  # Augmentation pipeline that randomly crop a patch from current image.
        crop_size=cfg.crop_size,  # The crop size of patch.
        cat_max_ratio=0.75),  # The max area ratio that could be occupied by single category.
    dict(
        type='RandomFlip',  # Augmentation pipeline that flip the images and their annotations
        flip_ratio=0.5),  # The ratio or probability to flip
    dict(type='PhotoMetricDistortion'),  # Augmentation pipeline that distort current image with several photo metric methods.
    dict(
        type='Normalize',  # Augmentation pipeline that normalize the input images
        **cfg.img_norm_cfg),
    dict(type='Pad',  # Augmentation pipeline that pad the image to specified size.
        size=cfg.crop_size,  # The output size of padding.
        pad_val=0,  # The padding value for image.
        seg_pad_val=255),  # The padding value of 'gt_semantic_seg'.
    dict(type='DefaultFormatBundle'),  # Default format bundle to gather data in the pipeline
    dict(type='Collect',  # Pipeline that decides which keys in the data should be passed to the segmentor
        keys=['img', 'gt_semantic_seg'])
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),  # First pipeline to load images from file path
    dict(
        type='MultiScaleFlipAug',  # An encapsulation that encapsulates the test time augmentations
        img_scale=(2048, 512),  # Decides the largest scale for testing, used for the Resize pipeline
        flip=False,  # Whether to flip images during testing
        transforms=[
            dict(type='Resize',  # Use resize augmentation
                 keep_ratio=True),  # Whether to keep the ratio between height and width, the img_scale set here will be suppressed by the img_scale set above.
            dict(type='RandomFlip'),  # Thought RandomFlip is added in pipeline, it is not used when flip=False
            dict(
                type='Normalize',  # Normalization config, the values are from img_norm_cfg
                 **cfg.img_norm_cfg),
            dict(type='ImageToTensor', # Convert image to tensor
                keys=['img']),
            dict(type='Collect', # Collect pipeline that collect necessary keys for testing.
                keys=['img'])
        ])
]
cfg.data = dict(
    samples_per_gpu=8,
    workers_per_gpu=8,
    train=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        img_dir=f'{img_dir}/training',
        ann_dir=f'{ann_dir}/training',
        pipeline=cfg.train_pipeline),
    val=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        img_dir=f'{img_dir}/validation',
        ann_dir=f'{ann_dir}/validation',
        pipeline=cfg.test_pipeline),
    test=dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        img_dir=f'{img_dir}/validation',
        ann_dir=f'{ann_dir}/validation',
        pipeline=cfg.test_pipeline))
############ end dataset

In [19]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor

# Build the dataset
datasets = [build_dataset(cfg.data.train)]
#print(datasets[0].CLASSES)
print(len(datasets[0].CLASSES))

2022-03-28 16:03:40,758 - mmseg - INFO - Loaded 5569 images


2


In [20]:
# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = len(datasets[0].CLASSES)

cfg.model.auxiliary_head[0].num_classes = len(datasets[0].CLASSES)
cfg.model.auxiliary_head[1].num_classes = len(datasets[0].CLASSES)
cfg.model.auxiliary_head[2].num_classes = len(datasets[0].CLASSES)

In [21]:
# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/teststdc2.Training_Tricks.160000iter.tensorboard'

############ start default runtime
cfg.log_config = dict(  # config to register logger hook
    interval=50,  # Interval to print the log
    hooks=[
        dict(type='TensorboardLoggerHook'),
        dict(type='TextLoggerHook', by_epoch=False)
    ])
cfg.dist_params = dict(backend='nccl')  # Parameters to setup distributed training, the port can also be set.
cfg.log_level = 'INFO'  # The level of logging.
cfg.load_from = None  # load models as a pre-trained model from a given path. This will not resume training.
cfg.resume_from = None  # Resume checkpoints from a given path, the training will be resumed from the iteration when the checkpoint's is saved.
cfg.workflow = [('train', 1)]  # Workflow for runner. [('train', 1)] means there is only one workflow and the workflow named 'train' is executed once. The workflow trains the model by 40000 iterations according to the `runner.max_iters`.
cfg.cudnn_benchmark = True  # Whether use cudnn_benchmark to speed up, which is fast for fixed input size.
############ end default runtime

############ start schedule
cfg.optimizer = dict(  # Config used to build optimizer, support all the optimizers in PyTorch whose arguments are also the same as those in PyTorch
    type='SGD',  # Type of optimizers, refer to https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/optimizer/default_constructor.py#L13 for more details
    lr=0.005,  # Learning rate of optimizers, see detail usages of the parameters in the documentation of PyTorch
    momentum=0.9,  # Momentum
    weight_decay=0.0005
    #paramwise_cfg=dict(custom_keys=dict(head=dict(lr_mult=10.0)))
)
cfg.optimizer_config = dict()  # Config used to build the optimizer hook, refer to https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/optimizer.py#L8 for implementation details.
lr_config = dict(
    policy='poly',  # The policy of scheduler, also support Step, CosineAnnealing, Cyclic, etc. Refer to details of supported LrUpdater from https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/lr_updater.py#L9.
    power=0.9,  # The power of polynomial decay.
    min_lr=0.00001,  # The minimum learning rate to stable the training.
    by_epoch=False,
    warmup='linear',
    warmup_iters=1000
)  # Whether count by epoch or not.
cfg.runner=dict(
    type='IterBasedRunner', # Type of runner to use (i.e. IterBasedRunner or EpochBasedRunner)
    max_iters=160000) # Total number of iterations. For EpochBasedRunner use `max_epochs`
cfg.checkpoint_config = dict(  # Config to set the checkpoint hook, Refer to https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/checkpoint.py for implementation.
    by_epoch=False,  # Whether count by epoch or not.
    interval=10000,
    meta=dict(
        CLASSES=classes,
        PALETTE=palette
    )
)
cfg.evaluation = dict(  # The config to build the evaluation hook. Please refer to mmseg/core/evaluation/eval_hook.py for details.
    interval=2000,  # The interval of evaluation.
    metric='mIoU',
    pre_eval=True)  # The evaluation metric.

############ end schedule

In [22]:
# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='BN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained=None,
    backbone=dict(
        type='STDCContextPathNet',
        backbone_cfg=dict(
            type='STDCNet',
            stdc_type='STDCNet2',
            in_channels=3,
            channels=(32, 64, 256, 512, 1024),
            bottleneck_type='cat',
            num_convs=4,
            norm_cfg=dict(type='BN', requires_grad=True),
            act_cfg=dict(type='ReLU'),
            with_final_conv=False),
        last_in_channels=(1024, 512),
        out_channels=128,
        ffm_cfg=dict(in_channels=384, out_channels=256, scale_factor=4)),
    decode_head=dict(
        type='FCNHead',
        in_channels=256,
        channels=256,
        num_convs=1,
        num_classes=2,
        in_index=3,
        concat_input=False,
        dropout_ratio=0.1,
        norm_cfg=dict(type='BN', requires_grad=True),
        align_corners=True,
        sampler=dict(type='OHEMPixelSampl

In [23]:
# Build the detector
model = build_segmentor(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

In [24]:
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES
model.PALETTE = datasets[0].PALETTE

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

In [25]:
train_segmentor(model, datasets, cfg, distributed=False, validate=True, meta=dict())

2022-03-28 16:03:42,003 - mmseg - INFO - Loaded 610 images
2022-03-28 16:03:42,003 - mmseg - INFO - Start running, host: user@ESWS-VT02-01, work_dir: /home/user/workspace/github/mmsegmentation/demo/work_dirs/teststdc2.Training_Tricks.160000iter
2022-03-28 16:03:42,004 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
 --------

2022-03-28 16:13:43,704 - mmseg - INFO - Iter [850/160000]	lr: 8.600e-03, eta: 1 day, 7:17:02, time: 0.728, data_time: 0.005, memory: 2073, decode.loss_ce: 0.1388, decode.acc_seg: 92.9015, aux_0.loss_ce: 0.1366, aux_0.acc_seg: 92.9270, aux_1.loss_ce: 0.1364, aux_1.acc_seg: 92.9270, aux_2.loss_ce: 0.0404, aux_2.loss_dice: 0.4026, aux_2.acc_seg: 99.2255, loss: 0.8547
2022-03-28 16:14:20,036 - mmseg - INFO - Iter [900/160000]	lr: 9.045e-03, eta: 1 day, 7:19:15, time: 0.727, data_time: 0.005, memory: 2073, decode.loss_ce: 0.1555, decode.acc_seg: 91.8150, aux_0.loss_ce: 0.1548, aux_0.acc_seg: 91.7926, aux_1.loss_ce: 0.1501, aux_1.acc_seg: 91.8158, aux_2.loss_ce: 0.0454, aux_2.loss_dice: 0.4246, aux_2.acc_seg: 99.0945, loss: 0.9304
2022-03-28 16:14:56,344 - mmseg - INFO - Iter [950/160000]	lr: 9.491e-03, eta: 1 day, 7:21:07, time: 0.726, data_time: 0.005, memory: 2073, decode.loss_ce: 0.1347, decode.acc_seg: 92.5571, aux_0.loss_ce: 0.1378, aux_0.acc_seg: 92.5570, aux_1.loss_ce: 0.1354, aux_1

2022-03-28 16:27:37,324 - mmseg - INFO - Iter [2000/160000]	lr: 9.889e-03, eta: 1 day, 7:29:34, time: 0.673, data_time: 0.005, memory: 2073, decode.loss_ce: 0.1564, decode.acc_seg: 91.4471, aux_0.loss_ce: 0.1558, aux_0.acc_seg: 91.4471, aux_1.loss_ce: 0.1551, aux_1.acc_seg: 91.4471, aux_2.loss_ce: 0.0439, aux_2.loss_dice: 0.4016, aux_2.acc_seg: 99.1915, loss: 0.9128
2022-03-28 16:28:13,739 - mmseg - INFO - Iter [2050/160000]	lr: 9.886e-03, eta: 1 day, 7:29:40, time: 0.728, data_time: 0.005, memory: 2073, decode.loss_ce: 0.1451, decode.acc_seg: 92.1969, aux_0.loss_ce: 0.1527, aux_0.acc_seg: 92.1969, aux_1.loss_ce: 0.1474, aux_1.acc_seg: 92.1969, aux_2.loss_ce: 0.0407, aux_2.loss_dice: 0.3823, aux_2.acc_seg: 99.2562, loss: 0.8682
2022-03-28 16:28:52,676 - mmseg - INFO - Iter [2100/160000]	lr: 9.883e-03, eta: 1 day, 7:32:53, time: 0.779, data_time: 0.059, memory: 2073, decode.loss_ce: 0.1623, decode.acc_seg: 91.3079, aux_0.loss_ce: 0.1582, aux_0.acc_seg: 91.3079, aux_1.loss_ce: 0.1584, au

2022-03-28 16:41:36,274 - mmseg - INFO - Iter [3150/160000]	lr: 9.824e-03, eta: 1 day, 7:27:12, time: 0.726, data_time: 0.006, memory: 2073, decode.loss_ce: 0.1315, decode.acc_seg: 93.0293, aux_0.loss_ce: 0.1331, aux_0.acc_seg: 93.0293, aux_1.loss_ce: 0.1260, aux_1.acc_seg: 93.0293, aux_2.loss_ce: 0.0414, aux_2.loss_dice: 0.3793, aux_2.acc_seg: 99.2400, loss: 0.8113
2022-03-28 16:42:12,635 - mmseg - INFO - Iter [3200/160000]	lr: 9.822e-03, eta: 1 day, 7:26:50, time: 0.728, data_time: 0.006, memory: 2073, decode.loss_ce: 0.1561, decode.acc_seg: 91.7844, aux_0.loss_ce: 0.1512, aux_0.acc_seg: 91.7844, aux_1.loss_ce: 0.1651, aux_1.acc_seg: 91.7844, aux_2.loss_ce: 0.0403, aux_2.loss_dice: 0.3655, aux_2.acc_seg: 99.2565, loss: 0.8782
2022-03-28 16:42:48,976 - mmseg - INFO - Iter [3250/160000]	lr: 9.819e-03, eta: 1 day, 7:26:26, time: 0.727, data_time: 0.006, memory: 2073, decode.loss_ce: 0.1274, decode.acc_seg: 92.9961, aux_0.loss_ce: 0.1330, aux_0.acc_seg: 92.9961, aux_1.loss_ce: 0.1301, au

2022-03-28 16:55:32,428 - mmseg - INFO - Iter [4300/160000]	lr: 9.760e-03, eta: 1 day, 7:16:58, time: 0.727, data_time: 0.005, memory: 2073, decode.loss_ce: 0.1301, decode.acc_seg: 92.8504, aux_0.loss_ce: 0.1310, aux_0.acc_seg: 92.8504, aux_1.loss_ce: 0.1326, aux_1.acc_seg: 92.8504, aux_2.loss_ce: 0.0371, aux_2.loss_dice: 0.3583, aux_2.acc_seg: 99.2848, loss: 0.7891
2022-03-28 16:56:08,826 - mmseg - INFO - Iter [4350/160000]	lr: 9.757e-03, eta: 1 day, 7:16:30, time: 0.728, data_time: 0.005, memory: 2073, decode.loss_ce: 0.1269, decode.acc_seg: 93.2525, aux_0.loss_ce: 0.1262, aux_0.acc_seg: 93.2525, aux_1.loss_ce: 0.1306, aux_1.acc_seg: 93.2525, aux_2.loss_ce: 0.0364, aux_2.loss_dice: 0.3455, aux_2.acc_seg: 99.3036, loss: 0.7656
2022-03-28 16:56:45,213 - mmseg - INFO - Iter [4400/160000]	lr: 9.755e-03, eta: 1 day, 7:16:02, time: 0.728, data_time: 0.005, memory: 2073, decode.loss_ce: 0.1400, decode.acc_seg: 92.6310, aux_0.loss_ce: 0.1381, aux_0.acc_seg: 92.6310, aux_1.loss_ce: 0.1356, au

...

2022-03-29 11:18:43,248 - mmseg - INFO - per class results:
2022-03-29 11:18:43,249 - mmseg - INFO - 
+------------+-------+-------+
|   Class    |  IoU  |  Acc  |
+------------+-------+-------+
| background | 96.97 | 98.84 |
|   human    | 53.27 | 64.62 |
+------------+-------+-------+
2022-03-29 11:18:43,249 - mmseg - INFO - Summary:
2022-03-29 11:18:43,249 - mmseg - INFO - 
+-------+-------+-------+
|  aAcc |  mIoU |  mAcc |
+-------+-------+-------+
| 97.07 | 75.12 | 81.73 |
+-------+-------+-------+
2022-03-29 11:18:43,252 - mmseg - INFO - Iter(val) [610]	aAcc: 0.9707, mIoU: 0.7512, mAcc: 0.8173, IoU.background: 0.9697, IoU.human: 0.5327, Acc.background: 0.9884, Acc.human: 0.6462
2022-03-29 11:19:19,612 - mmseg - INFO - Iter [95050/160000]	lr: 4.498e-03, eta: 13:09:39, time: 1.186, data_time: 0.465, memory: 2073, decode.loss_ce: 0.0724, decode.acc_seg: 95.5725, aux_0.loss_ce: 0.0715, aux_0.acc_seg: 95.5961, aux_1.loss_ce: 0.0744, aux_1.acc_seg: 95.3732, aux_2.loss_ce: 0.0326, aux_

2022-03-29 11:32:02,920 - mmseg - INFO - Iter [96100/160000]	lr: 4.434e-03, eta: 12:56:51, time: 0.674, data_time: 0.006, memory: 2073, decode.loss_ce: 0.0899, decode.acc_seg: 94.9228, aux_0.loss_ce: 0.0909, aux_0.acc_seg: 94.8726, aux_1.loss_ce: 0.0965, aux_1.acc_seg: 94.6223, aux_2.loss_ce: 0.0357, aux_2.loss_dice: 0.3152, aux_2.acc_seg: 99.2593, loss: 0.6282
2022-03-29 11:32:39,194 - mmseg - INFO - Iter [96150/160000]	lr: 4.431e-03, eta: 12:56:15, time: 0.726, data_time: 0.006, memory: 2073, decode.loss_ce: 0.0616, decode.acc_seg: 96.4033, aux_0.loss_ce: 0.0612, aux_0.acc_seg: 96.4012, aux_1.loss_ce: 0.0628, aux_1.acc_seg: 96.2851, aux_2.loss_ce: 0.0334, aux_2.loss_dice: 0.3090, aux_2.acc_seg: 99.3067, loss: 0.5280
2022-03-29 11:33:15,528 - mmseg - INFO - Iter [96200/160000]	lr: 4.428e-03, eta: 12:55:38, time: 0.727, data_time: 0.006, memory: 2073, decode.loss_ce: 0.0697, decode.acc_seg: 95.9675, aux_0.loss_ce: 0.0700, aux_0.acc_seg: 95.9270, aux_1.loss_ce: 0.0735, aux_1.acc_seg: 95

2022-03-29 11:45:58,453 - mmseg - INFO - Iter [97250/160000]	lr: 4.364e-03, eta: 12:42:50, time: 0.727, data_time: 0.006, memory: 2073, decode.loss_ce: 0.0953, decode.acc_seg: 94.8431, aux_0.loss_ce: 0.0969, aux_0.acc_seg: 94.7740, aux_1.loss_ce: 0.1001, aux_1.acc_seg: 94.6133, aux_2.loss_ce: 0.0350, aux_2.loss_dice: 0.3139, aux_2.acc_seg: 99.2598, loss: 0.6411
2022-03-29 11:46:27,072 - mmseg - INFO - Iter [97300/160000]	lr: 4.361e-03, eta: 12:42:09, time: 0.572, data_time: 0.006, memory: 2073, decode.loss_ce: 0.0646, decode.acc_seg: 96.1550, aux_0.loss_ce: 0.0644, aux_0.acc_seg: 96.1587, aux_1.loss_ce: 0.0685, aux_1.acc_seg: 95.9574, aux_2.loss_ce: 0.0327, aux_2.loss_dice: 0.3071, aux_2.acc_seg: 99.3132, loss: 0.5373
2022-03-29 11:47:03,422 - mmseg - INFO - Iter [97350/160000]	lr: 4.358e-03, eta: 12:41:32, time: 0.727, data_time: 0.005, memory: 2073, decode.loss_ce: 0.0712, decode.acc_seg: 96.0090, aux_0.loss_ce: 0.0704, aux_0.acc_seg: 96.0050, aux_1.loss_ce: 0.0730, aux_1.acc_seg: 95

2022-03-29 11:59:46,542 - mmseg - INFO - Iter [98400/160000]	lr: 4.293e-03, eta: 12:28:45, time: 0.727, data_time: 0.006, memory: 2073, decode.loss_ce: 0.0766, decode.acc_seg: 95.5998, aux_0.loss_ce: 0.0761, aux_0.acc_seg: 95.6208, aux_1.loss_ce: 0.0780, aux_1.acc_seg: 95.5200, aux_2.loss_ce: 0.0328, aux_2.loss_dice: 0.3059, aux_2.acc_seg: 99.3169, loss: 0.5694
2022-03-29 12:00:22,822 - mmseg - INFO - Iter [98450/160000]	lr: 4.290e-03, eta: 12:28:08, time: 0.725, data_time: 0.006, memory: 2073, decode.loss_ce: 0.0708, decode.acc_seg: 96.0549, aux_0.loss_ce: 0.0707, aux_0.acc_seg: 96.0353, aux_1.loss_ce: 0.0740, aux_1.acc_seg: 95.8372, aux_2.loss_ce: 0.0330, aux_2.loss_dice: 0.3150, aux_2.acc_seg: 99.3052, loss: 0.5636
2022-03-29 12:00:59,126 - mmseg - INFO - Iter [98500/160000]	lr: 4.287e-03, eta: 12:27:32, time: 0.726, data_time: 0.006, memory: 2073, decode.loss_ce: 0.0719, decode.acc_seg: 95.9355, aux_0.loss_ce: 0.0726, aux_0.acc_seg: 95.9492, aux_1.loss_ce: 0.0766, aux_1.acc_seg: 95

KeyboardInterrupt: 

In [ ]:
img = mmcv.imread('../data/ade/ADEChallengeData2016/images-binary/validation/ADE_val_00000011.jpg')
plt.imshow(img)

In [ ]:
img = mmcv.imread('../data/ade/ADEChallengeData2016/images-binary/validation/ADE_val_00000011.jpg')

model.cfg = cfg
result = inference_segmentor(model, img)

In [ ]:
np.asarray(result).max()

In [ ]:
plt.figure(figsize=(8, 6))
show_result_pyplot(model, img, result, palette)

In [ ]:
img = mmcv.imread('../data/ade/ADEChallengeData2016/binary-labels/validation/ADE_val_00000011.png')
img[img == 1] = 255
plt.imshow(img, cmap="gray")

In [ ]:
img.max()

In [ ]:
img.min()

In [ ]:
# print('show video')
# test a video and show the results
video = mmcv.VideoReader('../data/ramp.mp4')

In [ ]:
import time

In [ ]:
video[1].shape

In [ ]:
x = 100

In [ ]:
%%time
start = time.time()

for frame in video[:x]: # time - how many frame per sec - metrics acc - IoU
    result = inference_segmentor(model, frame)
    #model.show_result(frame, result)

end = time.time()

In [ ]:
from humanfriendly import format_timespan
print("Time:", format_timespan(end-start))
aux = (end-start)/len(video[:x])
print("Time/frame:", format_timespan(aux))
print("FPS: ", 1/aux)